In [26]:
import sympy as sym
class Iterative:

    def __init__(self, function):
        self.function = function

x , y = sym.symbols('x y')
f = sym.Function('f')
f

SyntaxError: cannot assign to function call (<ipython-input-26-a51ce380d4b0>, line 10)